## Relevance with Sparse Models

In [1]:
import config
import os
import pathlib
from dotenv import load_dotenv
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from src.data.s3_communication import S3Communication
from sparsezoo import Model
import zipfile
from io import BytesIO
import evaluate
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
import shutil
import torch
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

In [2]:
%env PYTORCH_NO_CUDA_MEMORY_CACHING=1

env: PYTORCH_NO_CUDA_MEMORY_CACHING=1


In [3]:
# Load credentials
dotenv_dir = os.environ.get(
    "CREDENTIAL_DOTENV_DIR", os.environ.get("PWD", "/opt/app-root/src")
)
dotenv_path = pathlib.Path(dotenv_dir) / "credentials.env"
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

In [4]:
# init s3 connector
s3c = S3Communication(
    s3_endpoint_url=os.getenv("S3_ENDPOINT"),
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    s3_bucket=os.getenv("S3_BUCKET"),
)

## Retrieve the test dataset and the trained models

In [5]:
s3c.download_files_in_prefix_to_dir(
    config.BASE_TRAIN_TEST_DATASET_S3_PREFIX,
    config.BASE_PROCESSED_DATA)

In [6]:
test_data_path = str(config.BASE_PROCESSED_DATA)+'/rel_test_split.csv'
test_data = pd.read_csv(test_data_path, index_col=0)
test_data.rename(columns={'text':'question', 'text_b':'sentence'}, inplace=True)

train_data_path = str(config.BASE_PROCESSED_DATA)+'/rel_train_split.csv'
train_data = pd.read_csv(train_data_path, index_col=0)
train_data.rename(columns={'text':'question', 'text_b':'sentence'}, inplace=True)

In [7]:
train_data[train_data['question']=='What is the annual total production from lignite (brown coal)?']

,label,question,sentence
403,0,What is the annual total production from ligni...,"PJM's Operating ORDC Filing — On March 29, 201..."
402,1,What is the annual total production from ligni...,64.8 million metric tons of lignite produced


In [8]:
trds = Dataset.from_pandas(train_data)
teds = Dataset.from_pandas(test_data.drop('label', axis=1))

climate_dataset = DatasetDict()

climate_dataset['train'] = trds
climate_dataset['test'] = teds

In [9]:
climate_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'question', 'sentence', '__index_level_0__'],
        num_rows: 2033
    })
    test: Dataset({
        features: ['question', 'sentence', '__index_level_0__'],
        num_rows: 509
    })
})

## Sparse model

In [10]:
test_data_path = str(config.BASE_PROCESSED_DATA)+'/rel_test_split.csv'
test_data = pd.read_csv(test_data_path, index_col=0)
test_data.rename(columns={'text':'question', 'text_b':'sentence'}, inplace=True)

In [11]:
num_labels = 2
batch_size = 16 #16

In [12]:
def save_model(local_path, model_name):
    trainer.save_model(local_path)
    shutil.make_archive(local_path, 'zip', local_path)
    buffer = BytesIO()
    with zipfile.ZipFile(buffer, 'a') as z:
        for dirname, _, files in os.walk(local_path):
            for f in files:
                f_path = os.path.join(dirname, f)
                with open (f_path, 'rb') as file_content:
                    z.writestr(f"{model_name}/{f}", file_content.read())
    buffer.seek(0)
    # upload model to s3
    s3c._upload_bytes(
        buffer_bytes=buffer,
        prefix=config.BASE_SAVED_MODELS_S3_PREFIX,
        key=f"{model_name}.zip"
    )

In [13]:
metric = evaluate.load('f1')


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)


def preprocess_function(examples):
    return tokenizer(examples['question'],
                     examples['sentence'],
                     truncation=True)

In [14]:
def scores_fn(groups):
    for group, data in groups:
        pred = data.pred
        true = data.label
        scores[group] = {}
        scores[group]["accuracy"] = accuracy_score(true, pred)
        scores[group]["f1_score"] = f1_score(true, pred)
        scores[group]["recall_score"] = recall_score(true, pred)
        scores[group]["precision_score"] = precision_score(true, pred)
        scores[group]["support"] = len(pred)
    scores_df = pd.DataFrame(scores)
    return scores_df

In [15]:
stubs=['zoo:nlp/text_classification/distilbert-none/pytorch/huggingface/mnli/pruned80_quant-none-vnni',
       'zoo:nlp/text_classification/distilbert-none/pytorch/huggingface/qqp/pruned80_quant-none-vnni',
       'zoo:nlp/text_classification/obert-base/pytorch/huggingface/mnli/pruned90_quant-none']

paths=['/opt/app-root/src/aicoe-osc-demo/models/distilbert_mnli_pruned80',
       '/opt/app-root/src/aicoe-osc-demo/models/distilbert_qqp_pruned80',
       '/opt/app-root/src/aicoe-osc-demo/models/obert_mnli_pruned90']

model_name_4_s3=['distilbert_mnli_pruned80', 'distilbert_qqp_pruned80', 'obert_mnli_pruned90']

In [16]:
model_scores = []

for (stub,path,model_name_s3) in zip(stubs, paths,model_name_4_s3):
    sparse_model = Model(stub, path)
    sparse_model.download()
    training_path=path + '/training'
    model_name = stub
    tokenizer = AutoTokenizer.from_pretrained(training_path, use_fast=True)
    sparse_model = AutoModelForSequenceClassification.from_pretrained(training_path, num_labels=num_labels)

    metric = evaluate.load('f1')
    encoded_climate_dataset = climate_dataset.map(preprocess_function, batched=True)

    args = TrainingArguments(
        f"{model_name}-finetuned",
        evaluation_strategy = "no",
        save_strategy = "no",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=16, #16
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='f1',
        push_to_hub = False,
        save_on_each_node=True
    )

    trainer = Trainer(
        sparse_model,
        args,
        train_dataset=encoded_climate_dataset["train"],
        eval_dataset=None,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    train = trainer.train()

    trainer.evaluate(eval_dataset=encoded_climate_dataset['test'])
    pred = trainer.predict(encoded_climate_dataset['test'])
    test_data["pred"] = np.argmax(pred.predictions, axis=1)
    groups = test_data.groupby("question")
    scores = {}

    # kpi wise performance metrics
    scores_df = scores_fn(groups)

    model_scores.append(
        {'Stub_Model': stub,
         'F1 Score': scores_df.loc['f1_score'].mean(),
         'train_runtime': train.metrics['train_runtime'],
         'train_samples_per_second':train.metrics['train_samples_per_second'],
         'train_steps_per_second':train.metrics['train_steps_per_second'],
         'total_flos':train.metrics['total_flos'],
         'train_loss':train.metrics['train_loss'],
         'epoch':train.metrics['epoch'],
         'test_runtime':pred.metrics['test_runtime'],
         'test_samples_per_second':pred.metrics['test_samples_per_second'],
         'test_steps_per_second':pred.metrics['test_steps_per_second']})
    #### Saving Model Locally and in S3###
    local_model_path = f'/opt/app-root/src/aicoe-osc-demo/models/transformers/{model_name_s3}'
    save_model(local_model_path, model_name_s3)
    torch.cuda.empty_cache()

downloading...:   0%|          | 0.00/198 [00:00<?, ?B/s]

downloading...:   0%|          | 0.00/655 [00:00<?, ?B/s]

downloading...:   0%|          | 0.00/8.47k [00:00<?, ?B/s]

downloading...:   0%|          | 0.00/3.68k [00:00<?, ?B/s]

downloading...:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: question, sentence, __index_level_0__. If question, sentence, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/app-root/lib64/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2033
  Num Epochs = 16


Step,Training Loss
500,0.363000
1000,0.155700
1500,0.084600
2000,0.054100




Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: question, sentence, __index_level_0__. If question, sentence, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 509
  Batch size = 16


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: question, sentence, __index_level_0__. If question, sentence, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 509
  Batch size = 16
/opt/app-root/lib64/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/app-root/lib64/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/a

In [17]:
df_model_score = pd.DataFrame(model_scores)

In [18]:
df_model_score

,Stub_Model,F1 Score,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss,epoch,test_runtime,test_samples_per_second,test_steps_per_second
0,zoo:nlp/text_classification/distilbert-none/py...,0.907176,239.9323,135.572,8.536,1.815870e+15,0.183508,16.0,1.2351,412.122,25.909
1,zoo:nlp/text_classification/distilbert-none/py...,0.913952,240.2301,135.404,8.525,1.815870e+15,0.179335,16.0,1.2394,410.689,25.819
2,zoo:nlp/text_classification/obert-base/pytorch...,0.895963,464.3408,70.052,4.411,3.606740e+15,0.161586,16.0,2.3217,219.234,13.783


# Conclusion

In this notebook, we compared three different models from the sparsezoo. The corresponding metrics were displayed in the dataframe above.